# <center> Libs

In [1]:
import os
import pandas as pd
import numpy as np
import json
from joblib import Parallel, delayed
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# <center> FUNCTIONS AREA

In [2]:
def split_description(x):
    local,date_time=x.split(' @ ')
    return local,date_time

def split_index(x):
    date,time=x.split(' ')
    time,trash=time.split('+')
    return date,time

def convert_column_json_to_dict(column):
    column=column.apply(lambda x:json.loads(x))
    return column

def extract_dfs(data):
    data['info']=convert_column_json_to_dict(data['info'])

    temp_dfs=[]
    for i in range(len(data['info'])):
        short_df = pd.DataFrame.from_dict(data['info'][i], orient='index').T
        temp_dfs.append(short_df)
    temp_df = pd.concat(temp_dfs, axis=0, ignore_index=True)
    temp_df.index=data['date']

    system_status=temp_df[['device','state']]
    system_status=system_status.dropna(subset=['device','state'])
    

    thp_sensors=temp_df[['temperature','linkquality','humidity','pressure']]
    thp_sensors=thp_sensors.dropna(subset=['temperature','linkquality'])


    dw_sensors=temp_df[['contact','linkquality','battery','voltage']]
    dw_sensors=dw_sensors.dropna(subset=['contact','linkquality'])


    motion_sensors=temp_df[['illuminance','linkquality','occupancy','battery','voltage']]
    motion_sensors=motion_sensors.dropna(subset=['illuminance','linkquality'])


    met_cond=temp_df[['pressure','windspeed','description','precipitation','winddirection','humidity','temperature']]
    met_cond=met_cond.dropna(subset=['windspeed','humidity','temperature'])


    met_cond[['local','date_time']]=met_cond['description'].apply(lambda x: pd.Series(split_description(x)))
    met_cond=met_cond.drop('description',axis=1)
    
    dfs=[system_status,thp_sensors,dw_sensors,motion_sensors,met_cond]
    
    return dfs

# <center> Fetching data

### Get the files

In [3]:
path='/Users/pedro/Desktop/Ironhack/Final-Project/dataverse_files/'
files=os.listdir(path)
files=[file for file in files if file.endswith('.csv')]
files

['sgh02015d5c61cc.csv',
 'sgh020149c615c5.csv',
 'sgh020125bce03a.csv',
 'sgh02019d93db3f.csv',
 'sgh0201a17a7a16.csv',
 'sgh02018fe9be2c.csv',
 'sgh020177a7a91d.csv',
 'sgh020102d29c86.csv',
 'sgh020114a6a800.csv']

### Create a dicionary to access the files more easily

In [4]:
file_key={}
for i,file in enumerate(files):
    name,trash=file.split('.')
    file_key[i]=name
file_key

{0: 'sgh02015d5c61cc',
 1: 'sgh020149c615c5',
 2: 'sgh020125bce03a',
 3: 'sgh02019d93db3f',
 4: 'sgh0201a17a7a16',
 5: 'sgh02018fe9be2c',
 6: 'sgh020177a7a91d',
 7: 'sgh020102d29c86',
 8: 'sgh020114a6a800'}

# <center> CLEANING AREA

### Create a dicionary to store the dfs grouped by files

In [ ]:
dfs_dict={}
for i, file in enumerate(files):
    name,trash=file.split('.')
    data=pd.read_csv(path+file)
    dfs_dict[name]=extract_dfs(data)



files_dict [file_key [ i ] ]   [ 0 ]   **system_status**

files_dict [file_key [ i ] ]   [ 1 ]   **thp_sensors**

files_dict [file_key [ i ] ]   [ 2 ]   **dw_sensors**

files_dict [file_key [ i ] ]   [ 3 ]   **motion_sensors**

files_dict [file_key [ i ] ]   [ 4 ]   **met_cond**



### Seperate the index (date-time) into columns date and time

In [ ]:
for i in range(0,9):
    for j in range(0,5):
        df=dfs_dict[file_key[i]][j]
        df['index_']=list(df.index)
        df[['date','time']]=df['index_'].apply(lambda x: pd.Series(split_index(x)))
        df.drop('index_', axis=1, inplace=True)
        df.reset_index(drop=True, inplace=True)
        if 'date_time' in df.columns:
            df.drop('date_time', axis=1, inplace=True)

### Correcting local names

In [ ]:
local=[]
for i in range(0,9):
    df=dfs_dict[file_key[i]][4]
    temp_list=df['local'].unique()
    for i in temp_list:
        if i not in local:
            local.append(i)
local

In [ ]:
correct_local={'Coimbra (Aeru00f3dromo)': 'Coimbra (Aeródromo)',
            'Lousu00e3 (Aeru00f3dromo)': 'Lousã (Aeródromo)',
            'Braganu00e7a': 'Bragança',
            'Chaves (Aeru00f3dromo)': 'Chaves (Aeródromo)'}

for i in range(0,9):
    df=dfs_dict[file_key[i]][4]
    df['local']=df['local'].replace(correct_local)

### Identify the dfs to concat them

In [ ]:
for i in range(0,9):
    for j in range(0,5):
        df=dfs_dict[file_key[i]][j]
        df.insert(loc=0, column='tenant', value=file_key[i])
        df.insert(loc=0, column='tenant_id', value=i)

In [ ]:
dfs_dict[file_key[8]][1]

### Concating the dfs by type

In [ ]:
system_status_dfs=[]
thp_sensors_dfs=[]
dw_sensors_dfs=[]
motion_sensors_dfs=[]
met_cond_dfs=[]

for i in range(0,9):
    system_status_dfs.append(dfs_dict[file_key[i]][0])
    thp_sensors_dfs.append(dfs_dict[file_key[i]][1])
    dw_sensors_dfs.append(dfs_dict[file_key[i]][2])
    motion_sensors_dfs.append(dfs_dict[file_key[i]][3])
    met_cond_dfs.append(dfs_dict[file_key[i]][4])   
    
    
system_status=pd.concat(system_status_dfs)
thp_sensors=pd.concat(thp_sensors_dfs)
dw_sensors=pd.concat(dw_sensors_dfs)
motion_sensors=pd.concat(motion_sensors_dfs)
met_cond=pd.concat(met_cond_dfs)

In [ ]:
#df=system_status.copy()
#df=thp_sensors.copy()
#df=dw_sensors.copy()
#df=motion_sensors.copy()
df=met_cond.copy()
df

# <center> EDA AREA

In [ ]:
def filter_date(df,date):
    days=df['date']
    return df[df['date'] == date]

def filter_df(df,column,filter_):
    return df[df[column] == filter_]

# Create functions to plot more easily

In [ ]:
def plotplot(df,x,y):
    fig=plt.plot(df[x], df[y])
    plt.xlabel(x)
    plt.ylabel(y)
    return plt.show()

def barplot(df,x,y):
    fig=plt.bar(df[x], df[y])
    plt.xlabel(x)
    plt.ylabel(y)
    return plt.show()

def scatterplot(df,x,y):
    df=df.dropna()
    fig=plt.scatter(df[x], df[y],color='blue',alpha=0.7)
    fig=plt.scatter(df[x], df[y],color='yellow',alpha=0.15)
    fig=plt.scatter(df[x], df[y],color='red',alpha=0.05)
    plt.xlabel(x)
    plt.ylabel(y)
    return plt.show()